In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from Feature_selecter import FeatureSelecter
from EMA_functions import *
import matplotlib.animation as animation
import pickle as pkl
from scipy.ndimage import gaussian_filter

file_name = "0_nad_1.cihx" #"measurement_0_00_degres" #"Full_web_ecc2_new_Floc7_v0"        #"0_nad_1.cihx"   # Second attempt, ecc2
EMA_structure = EMA_Structure(file_name)
video = EMA_structure.open_video(add_extension = False)
reference_image = (0, 100)#(0, 100)
bit_depth = 16

fig, ax = EMA_structure.plot_still_frame(video, reference_image)
mean_image = EMA_structure.mean_image
mean_image /= np.max(mean_image)
ani = EMA_structure.play_video(video, frame_range=range(200, 240))
video.info

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


D:/thijsmas/HSC - Ladisk\0_nad_1.cihx


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 40000, Original total frame: 77663)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


{'Date': '2021/6/21',
 'Camera Type': 'FASTCAM SA-Z type 2100K-M-64GB',
 'Record Rate(fps)': 20000.0,
 'Shutter Speed(s)': 20664.0,
 'Total Frame': 40000,
 'Original Total Frame': 77663,
 'Image Width': 768,
 'Image Height': 768,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Higher',
 'Color Bit': 16,
 'Comment Text': ''}

In [ ]:
# pixel = [690, 165]
# n = 11
# image0  = np.array(video.mraw[100])
# window0 = image0[pixel[1]-n//2:pixel[1]+n//2+1, pixel[0]-n//2:pixel[0]+n//2+1]
# image1  = np.array(video.mraw[101])
# window1 = image1[pixel[1]-n//2:pixel[1]+n//2+1, pixel[0]-n//2:pixel[0]+n//2+1]
# Ii, Ij = np.gradient(window0)
# Ix = Ij
# Iy = -Ii
# It = window1 - window0
# # IxIx = Ix*Ix
# # IyIy = Iy*Iy
# # IxIy = Ix*Iy
# # IxIt = Ix*It
# # IyIt = Iy*It
# Vx, Vy = np.zeros((n,n)), np.zeros((n,n))
# for pixel_i in range(n):
#     for pixel_j in range(n):
#         IxIx    = Ix[pixel_i, pixel_j]**2
#         IyIy    = Iy[pixel_i, pixel_j]**2
#         IxIy    = Ix[pixel_i, pixel_j]*Iy[pixel_i, pixel_j]
#         IxIt    = Ix[pixel_i, pixel_j]*It[pixel_i, pixel_j]
#         IyIt    = Iy[pixel_i, pixel_j]*It[pixel_i, pixel_j]
#         ATA = np.array([[IxIx, IxIy], [IxIy, IyIy]])
#         ATAinv = np.linalg.inv(ATA)
#         # denom = IxIx*IyIy - IxIy**2
#         # if denom == 0:
#         #     continue
#         # ATAinv = 1/denom * np.array([[IyIy, -IxIy], [-IxIy, IxIx]])
#         Vx[pixel_i, pixel_j], Vy[pixel_i, pixel_j] = -ATAinv @ np.array([IxIt, IyIt])
# X, Y = np.meshgrid(range(n), range(n))
# fig, ax = plt.subplots(2,1)
# ax[0].imshow(image0, cmap = "gray")
# ax[0].plot(pixel[0], pixel[1], "r+")
# ax[1].imshow(window1, cmap = "gray")
# ax[1].quiver(X, Y, Vx, Vy, color = "r")


In [ ]:
roi_size = (11,11)
im1 = np.mean(video.mraw[200:205,100:400,600:800], axis = 0)
# im1 = video.mraw[200,100:400,600:800]
# im1 = gaussian_filter(im1, 2)
im2 = np.mean(video.mraw[210:215,100:400,600:800], axis = 0)
# im2 = video.mraw[201,100:400,600:800]
# im2 = gaussian_filter(im2, 2)
feature_selecter = FeatureSelecter(im2)
feature_selecter.set_gt(im1, im2)
feature_selecter.set_filter_method('Vx', roi_size=roi_size)
Vx = feature_selecter.apply_filter()
feature_selecter.set_filter_method('Vy', roi_size=roi_size)
Vy = feature_selecter.apply_filter()

fig, ax = plt.subplots(1,5, figsize = (15,5))
ax[0].imshow(im1, cmap = "gray")
ax[1].imshow(im2, cmap = "gray")
ax[2].imshow(feature_selecter.gx, cmap = "gray")
ax[3].imshow(feature_selecter.gy, cmap = "gray")
ax[4].imshow(feature_selecter.gt, cmap = "gray")

Select points

In [ ]:
X, Y = np.meshgrid(range(0, video.mraw[0,100:400,600:800].shape[1]), range(0, video.mraw[0,100:400,600:800].shape[0]))
fig, ax = plt.subplots()
ax.imshow(im2-im1, cmap='gray')
ax.quiver(X, Y, Vx, Vy, color='r', scale=10)

In [2]:
min_distance = 15
roi_size_vec = [(3,3)]#, (5,5), (7,7), (9,9), (11,11)]
top_n_points = 24
feature_selecter = FeatureSelecter(mean_image)
for roi_size in roi_size_vec:
    feature_selecter.set_filter_method('eig0', roi_size=roi_size)
    scores = feature_selecter.apply_filter(downsample=1)
    points = feature_selecter.pick_max_filter(min_distance=min_distance, top_n_points = top_n_points)
    video.set_points(points)
    video.show_points()
    video.set_method('lk')
    video.method.configure(reference_image = reference_image, roi_size= roi_size)


Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test


c:\Users\thijsmas\Documents\GitHub\pyidi\Feature_selecter.py:236: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  ATA00 = pixel_list[::2] @ pixel_list[::2]  # equivalent to ATA[0, 0]
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\debugpy\_vendored\pydevd\pydev_ipython\qt_loaders.py:115: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\debugpy\_vendored\pydevd\pydev_ipython\qt_loaders.py:120: ResourceWarning: unclosed file <_io.BufferedReader name='c:\\Users\\thijsmas\\AppData\\Local\\anaconda3\\Lib\\site-packages\\PyQt5\\QtCore.pyd'>
  imp.find_module('QtCore', mod.__path__)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\debugpy\_vendored\pydevd\pydev_ipython\qt_

In [ ]:
roi_size = (3,3)
min_distance = 15
threshold_percentage = 80
feature_selecter = FeatureSelecter(mean_image)
feature_selecter.set_filter_method('eig0', roi_size=roi_size)
score_eig0_33 = feature_selecter.apply_filter(downsample=1)
points_eig0_33 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

feature_selecter.set_theta(90)
feature_selecter.set_filter_method('eig_theta', roi_size=roi_size)
score_eig90_33 = feature_selecter.apply_filter(downsample=1)
points_eig90_33 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

feature_selecter.set_theta(0)
feature_selecter.set_filter_method('eig_theta', roi_size=roi_size)
score_eig0_33 = feature_selecter.apply_filter(downsample=1)
points_eig_0_33 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

roi_size = (5,5)
feature_selecter.set_filter_method('eig0', roi_size=roi_size)
score_eig0_55 = feature_selecter.apply_filter(downsample=1)
points_eig0_55 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

feature_selecter.set_theta(90)
feature_selecter.set_filter_method('eig_theta', roi_size=roi_size)
score_eig90_55 = feature_selecter.apply_filter(downsample=1)
points_eig90_55 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

feature_selecter.set_theta(0)
feature_selecter.set_filter_method('eig_theta', roi_size=roi_size)
score_eig0_55 = feature_selecter.apply_filter(downsample=1)
points_eig_0_55 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)

roi_size = (11,11)
min_distance = 11
threshold_percentage = 70
feature_selecter = FeatureSelecter(mean_image)
feature_selecter.set_filter_method('eig0', roi_size=roi_size)
score_eig0_1111 = feature_selecter.apply_filter(downsample=1)
points_eig0_1111 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)
video.set_points(points_eig0_1111)
video.show_points()


In [ ]:
roi_size = (11,11)
min_distance = 11
threshold_percentage = 50
feature_selecter = FeatureSelecter(mean_image)
feature_selecter.set_filter_method('eig0', roi_size=roi_size)
score_eig0_1111 = feature_selecter.apply_filter(downsample=1)
points_eig0_1111 = feature_selecter.pick_max_filter(min_distance=min_distance, threshold_percentage=threshold_percentage)
video.set_points(points_eig0_1111)
video.show_points()

In [ ]:
video.set_method('lk')
video.method.configure(reference_image = reference_image, roi_size= roi_size, mraw_range=(1, 1000))
d_eig0_1111 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig0_1111.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig0_1111,d_eig0_1111), file)

In [ ]:
video.set_points(points_eig0_33)
video.show_points()
video.set_points(points_eig90_33)
video.show_points()
video.set_points(points_eig_0_33)
video.show_points()
video.set_points(points_eig0_55)
video.show_points()
video.set_points(points_eig90_55)
video.show_points()
video.set_points(points_eig_0_55)
video.show_points()

Extact motion

In [ ]:

video.set_method('lk')
video.method.configure(reference_image = reference_image, roi_size= (3, 3))

video.set_points(points_eig0_33)
d_eig0_33 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig0_33.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig0_33,d_eig0_33), file)

video.set_points(points_eig90_33)
d_eig90_33 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig90_33.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig90_33, d_eig90_33), file)

video.set_points(points_eig_0_33)
d_eig_0_33 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig_0_33.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig_0_33,d_eig_0_33), file)

video.method.configure(reference_image = reference_image, roi_size= (5, 5))
video.set_points(points_eig0_55)
d_eig0_55 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig0_55.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig0_55, d_eig0_55), file)

video.set_points(points_eig90_55)
d_eig90_55 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig90_55.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig90_55, d_eig90_55), file)

video.set_points(points_eig_0_55)
d_eig_0_55 = video.get_displacements(processes = 12)
pickle_file_path = "d_eig_0_55.pkl"
with open(pickle_file_path, "wb") as file:
    pkl.dump((points_eig_0_55, d_eig_0_55), file)

In [ ]:
%matplotlib qt
# valid_points_eig0_33 = (np.max(d_eig0_33, axis = (1, 2)) > .01) & (np.max(d_eig0_33, axis = (1, 2)) < 20)
# tp_eig0_33 = points_eig0_33[:, np.newaxis,:] + d_eig0_33
# EMA_structure.play_video(video, frame_range=range(1, 1000), points=tp_eig0[valid_points_eig0_33])

# valid_points_eig0_55 = (np.max(d_eig0_55, axis = (1, 2)) > .01) & (np.max(d_eig0_55, axis = (1, 2)) < 20)
# tp_eig0_55 = points_eig0_55[:, np.newaxis,:] + d_eig0_55
# EMA_structure.play_video(video, frame_range=range(1, 1000), points=tp_eig0_55[valid_points_eig0_55])

valid_points_eig0_1111 = (np.max(d_eig0_1111, axis = (1, 2)) > .01) & (np.max(d_eig0_1111, axis = (1, 2)) < 20)
tp_eig0_1111 = points_eig0_1111[:, np.newaxis,:] + d_eig0_1111
EMA_structure.play_video(video, frame_range=range(1, 1000), points=tp_eig0_1111[valid_points_eig0_1111])

# valid_points_eigth = (np.max(d_eigth, axis = (1, 2)) > 1) & (np.max(d_eigth, axis = (1, 2)) < 20)
# tp_eigth = points_eigth[::5][:, np.newaxis,:] + d_eigth
# EMA_structure.play_video(video, frame_range=range(1, 1000), points=tp_eigth[valid_points_eigth])

In [ ]:
%matplotlib inline
fps = video.info['Record Rate(fps)']
t_vec = np.arange(1, 1000)/fps
fig, ax = plt.subplots()
ax.plot(t_vec, (d_eig0[valid_points_eig0, :, 0][::5,:]).T)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Magnitude (pixels)')
ax.set_title('Magnitude of Displacements')
plt.show()

fig, ax = plt.subplots()
ax.plot(t_vec, (d_eigth[valid_points_eigth, :, 0][::5,:]).T)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Magnitude (pixels)')
plt.show()



In [ ]:
with open('d_eig0_33.pkl', 'rb') as file:
    d_eig0_33 = pkl.load(file)
with open('d_eig90_33.pkl', 'rb') as file:
    d_eig90_33 = pkl.load(file)
with open('d_eig_0_33.pkl', 'rb') as file:
    d_eig_0_33 = pkl.load(file)
with open('d_eig0_55.pkl', 'rb') as file:
    d_eig0_55 = pkl.load(file)
with open('d_eig90_55.pkl', 'rb') as file:
    d_eig90_55 = pkl.load(file)
with open('d_eig_0_55.pkl', 'rb') as file:
    d_eig_0_55 = pkl.load(file)
steps = 100
fps = video.info['Record Rate(fps)']
ylim = .250
xlim = .05
t_vec = np.arange(d_eig0_55.shape[1])/fps
fig, ax = plt.subplots(5,1, figsize = (15, 10))
ax[0].plot(t_vec, d_eig0_33[::steps,:,1].T)
ax[0].set_title('Eig0 3x3')
ax[1].plot(t_vec, d_eig90_33[::steps,:,1].T)
ax[1].set_title('Eig theta 90 3x3')
ax[2].plot(t_vec, d_eig_0_33[::steps,:,1].T)
ax[2].set_title('Eig theta 0 3x3')
ax[3].plot(t_vec, d_eig0_55[::steps,:,1].T)
ax[3].set_title('Eig0 5x5')
ax[4].plot(t_vec, d_eig90_55[::steps,:,1].T)
ax[4].set_title('Eig theta 90 5x5')
for ax_i in ax:
    ax_i.set_ylim(-ylim, ylim)
    ax_i.set_xlim(0, xlim)
    ax_i.set_xlabel('Time (s)')
    ax_i.set_ylabel('Magnitude (pixels)')


In [ ]:
fps = video.info['Record Rate(fps)']
t_vec = np.arange(d_eig0_1111.shape[1])/fps
n = d_eig0_1111.shape[1]
Freq = np.fft.rfftfreq(n, 1/fps)
Disp_FFT = np.fft.rfft(d_eig0_1111[:,:,1], n=n) / n
Acquisition_period  = t_vec[-1]
S_xx = 1/Acquisition_period * np.conj(Disp_FFT) * Disp_FFT
z_scores = (np.mean(S_xx, axis=1) - np.mean(S_xx)) / np.std(S_xx, axis=1)
threshold = 1
valid = np.abs(z_scores) < threshold
print('n excluded: ', np.sum(~valid))
S_xx = S_xx[valid]

S_xx_mean = np.mean(S_xx, axis = 0)
fig, ax = plt.subplots()
ax.semilogy(Freq, S_xx.T,'k-', alpha = .5, lw = 0.2)
ax.semilogy(Freq, S_xx_mean,'r-')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Sxx (pixels^2/Hz)')
ax.set_xlim(0, 1000)
